In [ ]:
!pip install tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%matplotlib inline
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
import zipfile
from tqdm import tqdm
import random
import shutil

In [ ]:
import zipfile
file_name="/content/drive/MyDrive/TB_Chest_Radiography_Database"

Dividing data into train, test and validation folders


In [ ]:
file_name1="/content/drive/MyDrive/TB_Chest_Radiography_Database/Tuberculosis"
train_folder = os.path.join(os.fspath(os.path.realpath(file_name)),'train')
val_folder = os.path.join(os.fspath(os.path.realpath(file_name)),'eval')
test_folder = os.path.join(os.fspath(os.path.realpath(file_name)), 'test')

# Define a list of image extensions
image_extension= ['.png']

# Create a list of image filenames in 'data_path'
imgs_list = [entry for entry in os.listdir(file_name1) if os.path.isfile(os.path.join(file_name1,entry))]
#print(len([entry for entry in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, entry))]))
# Sets the random seed
random.seed(42)

# Shuffle the list of image filenames
random.shuffle(imgs_list)

# determine the number of images for each set
train_size = int(len(imgs_list) * 0.7)
val_size = int(len(imgs_list) * 0.15)
test_size = int(len(imgs_list) * 0.15)

In [ ]:
train_datagen = ImageDataGenerator(rescale=1.0/255.0)
validation_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

In [ ]:
train_generator=train_datagen.flow_from_directory(train_folder,batch_size=20,class_mode='binary',target_size=(50,50))
validation_generator=validation_datagen.flow_from_directory(val_folder,batch_size=20,class_mode='binary',target_size=(50,50))
test_generator=test_datagen.flow_from_directory(test_folder,batch_size=20,class_mode='binary',target_size=(50,50))

Found 2946 images belonging to 2 classes.
Found 631 images belonging to 2 classes.
Found 633 images belonging to 2 classes.


In [ ]:
print(val_size)

105


In [ ]:
# Create destination folders if they don't exist
for folder_path in [train_folder, val_folder, test_folder]:
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

# Copy image files to destination folders
for i, f in enumerate(imgs_list):
    if i < train_size:
        dest_folder = train_folder
    elif i < train_size + val_size:
        dest_folder = val_folder
    else:
        dest_folder = test_folder
    shutil.copy(os.path.join(file_name1, f), os.path.join(dest_folder, f))

In [ ]:
file_name2="/content/drive/MyDrive/TB_Chest_Radiography_Database/Normal"
imgs_list1 = [entry for entry in os.listdir(file_name2) if os.path.isfile(os.path.join(file_name2,entry))]
random.seed(42)
random.shuffle(imgs_list1)
train_size1 = int(len(imgs_list1) * 0.7)
val_size1 = int(len(imgs_list1) * 0.15)
test_size1 = int(len(imgs_list1) * 0.15)
print(train_size1)

2457


In [ ]:
for i, f in enumerate(imgs_list1):
    if i < train_size1:
        dest_folder = train_folder
    elif i < train_size1 + val_size1:
        dest_folder = val_folder
    else:
        dest_folder = test_folder
    shutil.copy(os.path.join(file_name2, f), os.path.join(dest_folder, f))

In [ ]:
print(test_size1)

526


VGG19 model

In [ ]:
# prompt: import libraries to work with neural networks
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
import sys
import codecs
import tensorflow as tf

In [ ]:
# prompt: load pre trained VGG19 model

from keras.applications.vgg19 import VGG19
base_model = VGG19(weights='imagenet', include_top=False,input_shape=(224, 224, 3))

In [ ]:
!pip install tensorflow==2.8.0
!pip install tensorflow-gpu==2.8.0
!pip install keras==2.8.0
!pip install keras-bert
!pip install keras-rectified-adam
!pip install --upgrade tensorflow
!pip install tensorflow-dtensor

In [ ]:
# prompt: restart the kernel

%reset -f


In [ ]:
from chardet import detect
from keras_radam import RAdam
from keras import backend as K
from keras_bert import load_trained_model_from_checkpoint

In [ ]:
# prompt: import tensorflow.keras

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
img_path='/content/drive/MyDrive/TB_Chest_Radiography_Database/Tuberculosis/Tuberculosis-1.png'
img = image.load_img(img_path,color_mode='rgb', target_size=(224, 224))
display(img)

VGG19 Base model

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = tf.keras.optimizers.SGD(lr=0.001), loss = 'binary_crossentropy', metrics = ['acc'])
model.summary()

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
'''
# Load pre-trained VGG19 model (include_top=False means excluding the original classification head)
# Add your new classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Global average pooling layer
x = Dense(256, activation='relu')(x)  # Add a dense layer with ReLU activation
predictions = Dense(1, activation='sigmoid')(x)  # Output layer with softmax activation

# Create a new model with the modified VGG19 base and your custom classification head
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers in the base VGG19 model to prevent them from being trained
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Display the model summary to check the architecture
model.summary()'''


making the images of correct size

In [ ]:
vgghist = model.fit(train_generator, validation_data = validation_generator, steps_per_epoch = 50, epochs = 15)

In [ ]:
test_data_dir='/content/drive/MyDrive/TB_Chest_Radiography_Database/test'

In [ ]:
vgghist2 = model.fit(train_generator, validation_data = validation_generator, steps_per_epoch = 50, epochs = 5)

Epoch 1/5
50/50 [==============================] - 619s 12s/step - loss: 0.3491 - accuracy: 0.8720 - val_loss: 0.2591 - val_accuracy: 0.9081
Epoch 2/5
50/50 [==============================] - 318s 6s/step - loss: 0.1937 - accuracy: 0.9219 - val_loss: 0.2539 - val_accuracy: 0.9176
Epoch 3/5
50/50 [==============================] - 190s 4s/step - loss: 0.1892 - accuracy: 0.9340 - val_loss: 0.2334 - val_accuracy: 0.9223
Epoch 4/5
50/50 [==============================] - 164s 3s/step - loss: 0.1479 - accuracy: 0.9490 - val_loss: 0.2070 - val_accuracy: 0.9366
Epoch 5/5
50/50 [==============================] - 115s 2s/step - loss: 0.1495 - accuracy: 0.9480 - val_loss: 0.1977 - val_accuracy: 0.9366


In [ ]:
from PIL import Image
from skimage import transform

In [ ]:
model.input_shape

(None, 224, 224, 3)

In [ ]:
image_path = os.path.join(test_folder, "Tuberculosis","Tuberculosis-527.png")
print(image_path)
image_data = tf.io.read_file(image_path)
image_tensor = tf.image.decode_image(image_data)
image_shape = image_tensor.shape
print(image_shape)

/content/drive/MyDrive/TB_Chest_Radiography_Database/test/Tuberculosis/Tuberculosis-527.png
(512, 512, 3)


In [ ]:
print(image_tensor)

In [ ]:
image_tensor = np.array(image_tensor)

In [ ]:
import cv2

resized_image = cv2.resize(image_tensor, (224, 224))
prediction = model.predict(np.expand_dims(resized_image, axis=0))
display(resized_image)

In [ ]:
def load(filename):
    np_image = Image.open(filename)
    np_image = np.array(np_image).astype('float32')/255.
    np_image = np.expand_dims(np_image, axis=0)
    return np_image

In [ ]:
image_path=os.path.join(test_folder,"Tuberculosis","Tuberculosis-527.png")
image=Image.open(image_path)
image=image.resize((224,224))
image=np.array(image)
image=image/255.0
prediction = model.predict(np.expand_dims(image,axis=0))
negative_misprediction=0
if prediction >= 0.5:
        negative_misprediction = negative_misprediction + 1
        print("Incorrect prediction")
else:
        print("Correct prediction")

In [ ]:
test_file="/content/drive/MyDrive/TB_Chest_Radiography_Database/test/Tuberculosis"
test_list1 = [entry for entry in os.listdir(test_file) if os.path.isfile(os.path.join(test_file,entry))]
test_file2="/content/drive/MyDrive/TB_Chest_Radiography_Database/test/Normal"
test_list2 = [entry for entry in os.listdir(test_file2) if os.path.isfile(os.path.join(test_file2,entry))]
print(test_list1)
print(test_list2)
test_list=[]
test_list=test_list1+test_list2
print(test_list)
random.shuffle(test_list)
print(test_list)

In [ ]:
# prompt: increase the IOPub data rate

%%javascript
IPython.notebook.kernel.execute('notebook.iopub_data_rate_limit = 1e10')


In [ ]:
!pip install Pillow

In [ ]:
negative_misprediction = 0
val_items = os.listdir('/content/drive/MyDrive/TB_Chest_Radiography_Database/test/Tuberculosis')
#train_items = os.listdir('/content/train')

val_img_array = []
# load the image and convert into
# numpy array
for i in range(len(val_items)):
  img_path=os.path.join('/content/drive/MyDrive/TB_Chest_Radiography_Database/test/Tuberculosis',val_items[i])
  img = Image.open(img_path).convert('RGB')
  image=img.resize((224,224))
  numpydata = np.asarray(image)
  image = numpydata / 255.0
  image=np.expand_dims(image,axis=0)
  val_img_array.append(image)
  random.shuffle(val_img_array)
  print(val_img_array)


In [ ]:
for image in val_img_array:#os.listdir(os.path.join(test_folder, "Tuberculosis")):
    #image_path=os.path.join(test_folder, "Tuberculosis","Tuberculosis-531.png")
    #image=Image.open(image_path)
    #image=image.resize((224,224))
    #image=np.array(image)
    #image=image/255.0
    #image=np.expand_dims(image,axis=0)
    prediction = model.predict(image)
    #prediction = model.predict(load(os.path.join(test_folder, "Tuberculosis", str(image))))
    if prediction >= 0.5:
        negative_misprediction = negative_misprediction + 1
        print("Incorrect prediction")
    else:
        print("Correct prediction")

In [ ]:
total_negative_test_images = len(os.listdir(os.path.join('/content/drive/MyDrive/TB_Chest_Radiography_Database/test/Tuberculosis')))
print("Total images in negative validation set are ", total_negative_test_images)
print("Total correct classifications are ", total_negative_test_images - negative_misprediction)
print("Total incorrect classifications are ", negative_misprediction)

Total images in negative validation set are  106
Total correct classifications are  106
Total incorrect classifications are  0


In [ ]:
positive_misprediction = 0
val_items1 = os.listdir('/content/drive/MyDrive/TB_Chest_Radiography_Database/test/Normal')
#train_items = os.listdir('/content/train')

val_img_array1 = []
# load the image and convert into
# numpy array
for i in range(len(val_items1)):
  img_path=os.path.join('/content/drive/MyDrive/TB_Chest_Radiography_Database/test/Normal',val_items1[i])
  img = Image.open(img_path).convert('RGB')
  image=img.resize((224,224))
  numpydata = np.asarray(image)
  image = numpydata / 255.0
  image=np.expand_dims(image,axis=0)
  val_img_array1.append(image)
  random.shuffle(val_img_array1)
  print(val_img_array1)


In [ ]:
for image in val_img_array1:#os.listdir(os.path.join(test_folder, "Tuberculosis")):
    #image_path=os.path.join(test_folder, "Tuberculosis","Tuberculosis-531.png")
    #image=Image.open(image_path)
    #image=image.resize((224,224))
    #image=np.array(image)
    #image=image/255.0
    #image=np.expand_dims(image,axis=0)
    prediction = model.predict(image)
    #prediction = model.predict(load(os.path.join(test_folder, "Tuberculosis", str(image))))
    if prediction >= 0.5:
        positive_misprediction = positive_misprediction + 1
        print("Incorrect prediction")
    else:
        print("Correct prediction")

In [ ]:
total_positive_test_images = len(os.listdir(os.path.join('/content/drive/MyDrive/TB_Chest_Radiography_Database/test/Normal')))
print("Total images in positive validation set are ", total_positive_test_images)
print("Total correct classifications are ", total_positive_test_images - positive_misprediction)
print("Total incorrect classifications are ", positive_misprediction)

In [ ]:
total_test_images = total_positive_test_images + total_negative_test_images
total_correct_classifications = (total_positive_test_images - positive_misprediction) + (total_negative_test_images - negative_misprediction)
total_incorrect_classifications = positive_misprediction + negative_misprediction

print("Total test images ", total_test_images)
print("Correctly classified test images ", total_correct_classifications)
print("Incorrectly classified test images ", total_incorrect_classifications)

print(f"Accuracy Percentage ({total_correct_classifications } / {total_test_images}) * 100 = ",
      (total_correct_classifications/total_test_images)*100)